    1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news
    Для парсинга использовать xpath. Структура данных должна содержать:
* название источника,
* наименование новости,
* ссылку на новость,
* дата публикации

    2)Сложить все новости в БД


In [491]:
from lxml import html
from requests import get
from pprint import pprint
import re
import datetime
from pymongo import MongoClient

In [492]:
client = MongoClient('127.0.0.1', 27017)
db = client['news_database']         # создаем базу данных

In [493]:
yandex_news = 'https://yandex.ru'
header= {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
response = get(yandex_news + '/news', headers = header)

dom = html.fromstring(response.text)

blocks = dom.xpath("//div[@class='page-content__fixed page-content__fixed_middle']/div/table//td")

vchera1 = block.xpath(".//div[@class='story__info']/div[@class='story__date']/text()")[0]
vchera = str(re.findall('вчера', vchera1))
print(vchera)

result = []

for block in blocks:
    data = {}
    resource = block.xpath(".//div[@class='story__info']/div[@class='story__date']/text()")[0][0:-5]
    title = block.xpath(".//h2/a/text()")[0]
    link = block.xpath(".//h2/a//@href")[0]
    time = block.xpath(".//div[@class='story__info']/div[@class='story__date']/text()")[0][-5:]
    
    data['resource'] = resource
    data['title'] = title
    data['link'] = yandex_news + link
    data['time'] = time
    data['date'] = str(datetime.date.today())
    result.append(data)

yandex_news = db.yandex_news  # создаем коллекцию

for i in result:
    yandex_news.update_one({'link':i['link']},{'$set':i},upsert=True)

for i in yandex_news.find({}).limit(3):
    pprint(i)

['вчера']
{'_id': ObjectId('5ec987e2e912c38a0ac91906'),
 'date': '2020-05-23',
 'link': 'https://yandex.ru/news/story/V_Moskve_proveli_bolee_50_tysyach_testov_na_antitela_k_koronavirusu--f63afce7c8f82b61f5883852301b897a?lr=132&lang=ru&stid=fOsYnhcEoGOWxNp65xXi&persistent_id=98404175&rubric=Moscow&from=index',
 'source': 'RT на русском ',
 'time': '22:28',
 'title': 'В Москве провели более 50 тысяч тестов на антитела к коронавирусу'}
{'_id': ObjectId('5ec987e2e912c38a0ac91908'),
 'date': '2020-05-24',
 'link': 'https://yandex.ru/news/story/Sinoptiki_priznali_majskie_kholoda_v_Moskve_anomalnymi--1c2616627e01230a98a18e7299db5c12?lr=132&lang=ru&stid=iUCf5CKboQ_g01trtACx&persistent_id=98428964&rubric=Moscow&from=index',
 'resource': 'Lenta.ru вчера\xa0в\xa0',
 'source': 'Lenta.ru ',
 'time': '22:27',
 'title': 'Синоптики признали майские холода в Москве аномальными'}
{'_id': ObjectId('5ec987e2e912c38a0ac9190a'),
 'date': '2020-05-23',
 'link': 'https://yandex.ru/news/story/Otkrylas_onlajn-z

In [494]:
lenta = 'https://lenta.ru/'

header= {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
response = get(lenta, headers = header)

dom_lenta = html.fromstring(response.text)

result_lenta = []

title_lenta = dom_lenta.xpath(".//time/ancestor::a/text()")
link_lenta = dom_lenta.xpath(".//time/ancestor::a/@href")
time_lenta = dom_lenta.xpath(".//time[@datetime]/text()")
date_lenta = dom_lenta.xpath(".//time/@title")
lenta = 'lenta.ru'
 
for i in range (len(title_lenta)):
    lenta_data = {}    
    lenta_data['resource'] = lenta
    lenta_data['title'] = title_lenta[i].replace('\xa0','')
    lenta_data['link'] = lenta + link_lenta[i]
    lenta_data['time'] = time_lenta[i]
    lenta_data['date'] = date_lenta[i]
    result_lenta.append(lenta_data)

    
lenta_news = db.lenta  # создаем коллекцию

for i in result_lenta:
    lenta_news.update_one({'link':i['link']},{'$set':i},upsert=True)

for i in lenta_news.find({}).limit(3):
    pprint(i)

{'_id': ObjectId('5ecab724e912c38a0ac97f02'),
 'date': '24 мая 2020',
 'link': 'lenta.ru/news/2020/05/24/ne_dojdetes/',
 'resource': 'lenta.ru',
 'time': '20:56',
 'title': 'В России ответили насообщения обусталости Путина отвойны вСирии'}
{'_id': ObjectId('5ecab724e912c38a0ac97f04'),
 'date': '24 мая 2020',
 'link': 'lenta.ru/news/2020/05/24/death_football/',
 'resource': 'lenta.ru',
 'time': '20:59',
 'title': 'Подсчитано количество смертей откоронавируса после посещения матча '
          'ЛЧ'}
{'_id': ObjectId('5ecab724e912c38a0ac97f06'),
 'date': '24 мая 2020',
 'link': 'lenta.ru/news/2020/05/24/vse_nepravda/',
 'resource': 'lenta.ru',
 'time': '20:39',
 'title': 'Тарзан ответил накритику из-за жалоб нанехватку денег'}


In [495]:
mail = 'https://news.mail.ru/'

header= {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
response = get(mail, headers = header)

dom_mail = html.fromstring(response.text)

result_mail = []

blocks_news_mail = dom_mail.xpath("//li/span/a/@href") #links


for i in blocks_news_mail:
    response_bnm = get(mail + i, headers = header)
    dom_mail_bnm = html.fromstring(response_bnm.text)
    
    mail_data = {}    
    mail_data['resource'] = dom_mail_bnm.xpath("//a[contains(@class,'mbs')]/span[@class='link__text']/text()")
    mail_data['title'] = dom_mail_bnm.xpath("//h1/text()")
    mail_data['link'] = mail + i
    mail_data['time'] = dom_mail_bnm.xpath("//span/@datetime")[0][11:16]
    mail_data['date'] = dom_mail_bnm.xpath("//span/@datetime")[0][:10]
    result_mail.append(mail_data)

    
mail_news = db.mail  # создаем коллекцию

for i in result_mail:
    mail_news.update_one({'link':i['link']},{'$set':i},upsert=True)

for i in mail_news.find({}).limit(3):
    pprint(i)

{'_id': ObjectId('5ecaef9fe912c38a0ac99886'),
 'date': '2020-05-24',
 'link': 'https://news.mail.ru//politics/41909070/',
 'resource': ['ТАСС'],
 'time': '17:03',
 'title': ['Нетаньяху заявил о готовности Израиля противостоять агрессии '
           'Ирана']}
{'_id': ObjectId('5ecaef9fe912c38a0ac99888'),
 'date': '2020-05-24',
 'link': 'https://news.mail.ru//politics/41908824/',
 'resource': ['РИА Новости'],
 'time': '16:38',
 'title': ['Захарова объяснила проблему Договора об открытом небе для США']}
{'_id': ObjectId('5ecaef9fe912c38a0ac9988a'),
 'date': '2020-05-24',
 'link': 'https://news.mail.ru//politics/41908165/',
 'resource': ['Коммерсантъ'],
 'time': '14:50',
 'title': ['Япония планирует полностью отменить режим ЧС с 25 мая']}
